In [ ]:
# -*- coding: utf-8 -*-
# -----------------------------------------------------------------------------
# Module name       : training of the ResNext model for audio
# Description       : Read Functions
# Author            : Francisco Gómez <fagomezj@unal.edu.co>, Freddy Hernández <fohernandezr@unal.edu.co>
# Creation date     : 2025
# -----------------------------------------------------------------------------

# Note: This notebook runs locally with using NVIDIA RTX-4090 GPU (24G)

# Generate samples using already trained diffusion models

Generate samples using already trained diffusion models for the different species

In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append('C:/Users/user/Documents/DataScience/AnuraSoundProject/Classification')
sys.path.append('C:/Users/user/Documents/DataScience/AnuraSoundProject/DataAugmentation/diffusion')
sys.path.append("C:/Users/user/Documents/DataScience/AnuraSoundProject/DataAugmentation/diffusion/model_utils")

In [5]:
########
import os
import shutil
import random
from pathlib import Path
########
from audio_classification import configureModel,saveModel
from transformers import AutoFeatureExtractor

########
import argparse
import json
import models
import training_loss
import torch
import dataset_manager
from dataset_manager import GenerativeAIDataset
from torch.utils.data import DataLoader
from transformation.data_preprocessing import get_data_composing
from tqdm import tqdm

from torch.utils.data import Dataset
import os
''' Load parameters like we did in the training script '''
from collections import namedtuple
from dataset_manager import GenerativeAIDataset


#########
import argparse
import json
import models
import training_loss
import torch
import dataset_manager
from dataset_manager import GenerativeAIDataset
from torch.utils.data import DataLoader
from transformation.data_preprocessing import get_data_composing
from tqdm import tqdm

from torch.utils.data import Dataset
import os
from collections import namedtuple

''' Now we can generate samples using the model '''
from sampling.sampling_utils import get_diffusion_sample
import sys
from model_utils.diffusion_utils import calc_diffusion_hyperparams
from dataset_manager import GenerativeAIDataset
import numpy as np
import librosa
import soundfile as sf
import os


def saveAudiosToFolder(audios,output_dir,sample_rate,nameFile):    
    """
    Saves a list of audio arrays to WAV files in the specified directory.

    Parameters:
    -----------
    audios : list or array-like
        A list or array of audio signals (each as a NumPy array). Each signal should be 1D or 2D (with channel dim).
    output_dir : str
        The directory where the audio files will be saved. Created if it does not exist.
    sample_rate : int
        The sample rate (in Hz) to use when saving the WAV files.
    nameFile : str
        A base name for the output files. Each file will be named as "{nameFile}_audio_{i}.wav".

    Notes:
    ------
    - Converts each audio signal to float32 if not already.
    - Uses `soundfile.write` to save each file.
    """
    # Create folder if is missing
    os.makedirs(output_dir, exist_ok=True)
    
    # Save the audio
    for i, audio in enumerate(audios):
        y = audio.squeeze()  
        
        # It is in a valid range?
        if y.dtype != np.float32:
            y = y.astype(np.float32)
    
        filename = os.path.join(output_dir, nameFile+f"_audio_{i+1}.wav")
        sf.write(filename, y, sample_rate)
    
    print("Generated Audios Saved")

C:\Users\user\anaconda3\envs\tf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\user\anaconda3\envs\tf\lib\site-packages\transformers\configuration_utils.py:314: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


Configuration for generate samples using the model

In [19]:
''' Now we can generate samples using the model '''
Args = namedtuple('Args', [
    'batch_size',
    'learning_rate',
    'max_epochs',
    'model',
    'config'
])
#############################
args = Args(
    batch_size=8,
    learning_rate=1e-3,
    max_epochs=350,
    model='DIFFUSION',
    config='./config.json')


pathDiffusionAugmentedData = "C:/Users/user/Documents/soundDBAugmented/generated_by_diffusion"
nsamplesToGenerate = 200
sr = 16000
labelsClasses = ['s1','s2','s3','s4','s5','s6','s7','s8','s9']